# Wild Drone LLM Workshop: Part 1 - Travel Assistant Agent

Outdated! - Updated version in Google Colab: 
https://colab.research.google.com/drive/11cgnpWTMLjIDlOyYQ23STOYHWmjZf5ii?usp=sharing

## Setup: Get Your API Key

You need a **free Gemini API key** from: https://aistudio.google.com/app/apikey

**Quick Setup:**
1. Copy `.env.example` to `.env`
2. Add your key: `GEMINI_API_KEY=your_key_here`
3. Run the cells below

**Alternative:** Set in terminal: `export GEMINI_API_KEY=your_key_here`

In [ ]:
# Install required packages
!pip install litellm matplotlib numpy python-dotenv

# Note: If using OpenAI instead of Gemini, also install:
# !pip install openai

In [ ]:
# Configuration and Setup
import os
import litellm
from dotenv import load_dotenv
from llm_agents import add_parameters_schema, LLMAgent

# Load environment variables from .env file (if it exists)
load_dotenv()

# MODEL CONFIGURATION
# Get Gemini API key from environment variable
gemini_api_key = os.getenv('GEMINI_API_KEY')
# Set the API key for litellm
os.environ['GEMINI_API_KEY'] = gemini_api_key
# Set the model to use throughout the workshop
model_name = "gemini/gemini-2.5-flash"  # Using stable model

# Understanding LLMs

## Your First LLM Conversation
Let's start simple - ask an LLM a question and see how it responds. 

## TO-DO 📝
- Try modifying the system prompt to see how it affects the response
- Try modifying the user prompt to ask different questions
- Try modifying the temperature, how does the response look as we try higher values like 0.5, 1.0 or 2.0?

In [ ]:
system_prompt = "You are a helpful assistant that answers with short poems."
user_prompt = "How do I make a Spanish tortilla?"

response = litellm.completion(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=2000, # maximum length of the response in tokens
        temperature=0.0, # 0.0 is deterministic (should always yield same response), more than 0.0 adds randomness and creativity
    )
print(response.choices[0].message.content)

## The Problem: LLMs Are Limited to their Training Data

LLMs can't:
- Get current weather 🌤️
- Access the internet 🌐

They will either say they don't have the requested information, or hallucinate a response.

## TO-DO 🤔💭
- Read and understand the system prompt
- Observe how the capabilities of the model are limited when we ask for up-to-date information

In [ ]:
# Let's create a travel agent with no tools
weather_uninformed_travel_agent = LLMAgent(model=model_name)

# Set an empty list of tools (no tools)
weather_uninformed_travel_agent.tools = []

# Define a system prompt for the travel agent
weather_uninformed_travel_agent.system_prompt = """
You are a helpful travel planning assistant.
Help users decide between travel destinations based on the weather forecast.
Answer in a friendly and concise manner (max 30 words).
"""

# Test the uninformed travel agent
question = "I'm hesitating between going to Paris or Rome for a 3-day trip next weekend. Which destination should I choose?"

print("🤖 Testing our Weather-Uninformed Travel Agent:")
print("-" * 50)
response = weather_uninformed_travel_agent.chat(question)

**Solution**: Give LLMs access to tools so they can get updated information from APIs such as weather forecasts.

## How Tool Calling Works

1. **Define tools** - Tell the LLM what functions it can use
2. **LLM chooses** - It decides which tool to call based on the user's question
3. **Get results** - The function runs and returns real data
4. **Smart response** - LLM uses the fresh data to give informed answers

Let's build a weather-informed travel agent! 🌦️✈️

We'll start by creating a mock function that provides weather forecasts for the next N days in any city.

> **Important Note**: The tools in this workshop are **mock functions** that generate random data for educational purposes. In real applications, these would connect to actual APIs like: OpenWeatherMap, Skyscanner, National railway APIs & Web search APIs to get current information

## TO-DO 🤔💭
- Read and understand what the weather tool is doing
- Pay attention to how the metadata (parameters schema and docstring) provide info to the model on how to use the tool
- Look at how the tool is connected to the LLM Agent
- Read and understand the system prompt of the LLM Agent
- Observe how now the agent can provided guidance on where to travel based on the weather data


In [ ]:
# Our First Tool: Weather Forecast

@add_parameters_schema( # Info for model about type and description of parameters
    city={"type": "string", "description": "City name"},
    days={"type": "integer", "description": "Number of days (1-7)"}
)
def get_weather_forecast(city: str, days: int = 3) -> str:
    """Get weather forecast for a city""" # Info for model about what the function does
    import random
    
    # Possible weather conditions
    conditions = ["Sunny", "Cloudy", "Rainy", "Snowy"]
    
    # Generate random forecast
    result = f"Weather for {city}:\n"
    for day in range(1, days + 1):
        weather = random.choice(conditions)
        temp = random.randint(5, 25)  # Temperature in Celsius
        result += f"Day {day}: {weather}, {temp}°C\n"
    
    return result

In [ ]:
# Test the tool: weather forecast for Paris for 3 days
weather_forecast = get_weather_forecast("Paris", 3)
print(weather_forecast)

In [ ]:
# Now let's create and test a travel agent that is weather-informed
weather_informed_travel_agent = LLMAgent(model=model_name)

# Connect the tools defined in the notebook
weather_informed_travel_agent.tools = [get_weather_forecast]

# Set the system prompt defined in the notebook
weather_informed_travel_agent.system_prompt = """
You are a helpful travel planning assistant.
Your task is to help users decide between travel destinations.
Use the tools provided to make an informed recommendation.
Answer in a friendly and concise manner (max 30 words).
"""
# Test our travel agent
question = """I'm hesitating between going to Paris or Rome for a 3-day trip next weekend. Which destination should I choose?"""

print("🤖 Testing our Weather-Informed Travel Agent:")
print("-" * 50)

# Get a response from the travel agent
response = weather_informed_travel_agent.chat(question)

# 🚀 Advanced Exercise: Compare Travel Options

Now let's add flight and train price checking! Your agent will help users choose between different travel options.

## Your Tasks 📝

1. **Study the tools below** - See how they work
2. **Write a system prompt** - Tell your agent how to help users  
3. **Connect the tools** - Add all tools to your agent
4. **Test it out** - Try different travel questions

The tools give realistic prices and CO2 emissions. Your job: make an agent that helps users choose wisely! 🌱💰

> **Remember**: These tools generate random data for learning purposes. Real travel agents would use live APIs to get actual prices, schedules, and availability from airlines, booking sites, and transportation providers.

In [ ]:
# Tool 1: Flight Prices
@add_parameters_schema(
    origin={"type": "string", "description": "Departure city"},
    destination={"type": "string", "description": "Destination city"}
)
def get_flight_prices(origin: str, destination: str) -> str:
    """Get flight prices and environmental impact"""
    import random
    
    # Simple price calculation
    price = random.randint(150, 400)
    airline = random.choice(["BudgetAir", "SkyConnect", "EuroWings"])
    
    # Flight details
    hours = random.randint(2, 8)
    co2_kg = random.randint(80, 200)  # CO2 emissions in kg
    
    return f"✈️ Flight {origin} → {destination}: {airline} €{price} ({hours}h, CO2: {co2_kg}kg)"

# Test it!
print(get_flight_prices("Paris", "Rome"))

In [ ]:
# Tool 2: Train Prices  
@add_parameters_schema(
    origin={"type": "string", "description": "Departure city"},
    destination={"type": "string", "description": "Destination city"}
)
def get_train_prices(origin: str, destination: str) -> str:
    """Get train prices and environmental impact"""
    import random
    
    # Some routes don't exist
    no_train_routes = [("London", "Rome"), ("Madrid", "Stockholm")]
    if (origin, destination) in no_train_routes or (destination, origin) in no_train_routes:
        return f"🚫 No train route from {origin} to {destination}"
    
    # Simple price calculation (usually cheaper than flights)
    price = random.randint(80, 250)
    operator = random.choice(["EuroRail", "HighSpeed", "RegionalTrain"])
    
    # Train details  
    hours = random.randint(4, 12)
    co2_kg = random.randint(10, 40)  # Much lower CO2 than flights!
    
    return f"🚂 Train {origin} → {destination}: {operator} €{price} ({hours}h, CO2: {co2_kg}kg)"

# Test it!
print(get_train_prices("Paris", "Rome"))

In [ ]:
# TODO: STUDENT TASK - Create and configure your enhanced agent
# 1. Create a new LLMAgent instance
enhanced_travel_agent = LLMAgent(model=model_name)

# 2. Add the weather, flight and trains tool to the agent

# 3. Write your enhanced system prompt here to consider cost and sustainability
enhanced_travel_agent.system_prompt = """
Write your system prompt here!

Consider including:
- Your agent's role and personality
- How to balance cost vs environmental impact  
- What factors to prioritize (price, CO2, time, comfort)
- How to structure recommendations
- Tone (friendly, professional, expert, etc.)
"""

## 🧪 Test Your Enhanced Agent

Try different questions, ensure the agent uses the tools to get updated information, and that it replies as expected.

**What to Notice:**
- Does it check all options (weather, flights, trains)?
- How does it explain trade-offs?
- Is the advice helpful and clear?

In [ ]:
# Test your enhanced travel agent here!
# Make sure you've completed the agent creation above first

test_question = """
I'm hesitating between traveling from Munich to Paris or Rome for a 3-day trip next weekend. Which destination should I choose?
"""

enhanced_travel_agent.chat(test_question)

## Provided solution

In [ ]:
# Provided solution to the advanced travel agent task
# 1. Create a new LLMAgent instance
enhanced_travel_agent = LLMAgent(model=model_name)
# 2. Add the weather, flight and trains tool to the agent
enhanced_travel_agent.tools = [get_weather_forecast, get_flight_prices, get_train_prices]
# 3. Write your enhanced system prompt here to consider cost and sustainability
enhanced_travel_agent.system_prompt = """
You are an expert travel planning assistant.
Your task is to help users decide between travel destinations.
Use the tools provided to make an informed recommendation.
Prioritize affordable and low-CO2 options, but also consider user comfort.
Answer in a friendly and concise manner (max 50 words).
"""

In [ ]:
# Test the provided solution
question = "I'm hesitating between traveling from Munich to Paris or Rome for a 3-day trip next weekend. Which destination should I choose?"

print("🤖 Testing our Enhanced Travel Agent:")
print("-" * 50)
response = enhanced_travel_agent.chat(question)